# Мое решение

In [1]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

In [2]:
train_df = pd.read_csv('/kaggle/input/playground-series-s4e7/train.csv')
test_df = pd.read_csv('/kaggle/input/playground-series-s4e7/test.csv')

In [3]:
train_df

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,0,Male,21,1,35.0,0,1-2 Year,Yes,65101.0,124.0,187,0
1,1,Male,43,1,28.0,0,> 2 Years,Yes,58911.0,26.0,288,1
2,2,Female,25,1,14.0,1,< 1 Year,No,38043.0,152.0,254,0
3,3,Female,35,1,1.0,0,1-2 Year,Yes,2630.0,156.0,76,0
4,4,Female,36,1,15.0,1,1-2 Year,No,31951.0,152.0,294,0
...,...,...,...,...,...,...,...,...,...,...,...,...
11504793,11504793,Male,48,1,6.0,0,1-2 Year,Yes,27412.0,26.0,218,0
11504794,11504794,Female,26,1,36.0,0,< 1 Year,Yes,29509.0,152.0,115,1
11504795,11504795,Female,29,1,32.0,1,< 1 Year,No,2630.0,152.0,189,0
11504796,11504796,Female,51,1,28.0,0,1-2 Year,Yes,48443.0,26.0,274,1


In [6]:
test_df

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
0,11504798,Female,20,1,47.0,0,< 1 Year,No,2630.0,160.0,228
1,11504799,Male,47,1,28.0,0,1-2 Year,Yes,37483.0,124.0,123
2,11504800,Male,47,1,43.0,0,1-2 Year,Yes,2630.0,26.0,271
3,11504801,Female,22,1,47.0,1,< 1 Year,No,24502.0,152.0,115
4,11504802,Male,51,1,19.0,0,1-2 Year,No,34115.0,124.0,148
...,...,...,...,...,...,...,...,...,...,...,...
7669861,19174659,Male,57,1,28.0,0,1-2 Year,Yes,51661.0,124.0,109
7669862,19174660,Male,28,1,50.0,1,< 1 Year,No,25651.0,152.0,184
7669863,19174661,Male,47,1,33.0,1,1-2 Year,No,2630.0,138.0,63
7669864,19174662,Male,30,1,28.0,0,< 1 Year,Yes,38866.0,124.0,119


In [10]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7669866 entries, 0 to 7669865
Data columns (total 11 columns):
 #   Column                Dtype  
---  ------                -----  
 0   id                    int64  
 1   Gender                object 
 2   Age                   int64  
 3   Driving_License       int64  
 4   Region_Code           float64
 5   Previously_Insured    int64  
 6   Vehicle_Age           object 
 7   Vehicle_Damage        object 
 8   Annual_Premium        float64
 9   Policy_Sales_Channel  float64
 10  Vintage               int64  
dtypes: float64(3), int64(5), object(3)
memory usage: 643.7+ MB


In [4]:
gender_mapping = {'Male': 0, 'Female': 1}
vehicle_age_mapping = {'< 1 Year': 0, '1-2 Year': 1, '> 2 Years': 2}
vehicle_damage_mapping = {'No': 0, 'Yes': 1}
        
train_df['Gender'] = train_df['Gender'].map(gender_mapping)
test_df['Gender'] = test_df['Gender'].map(gender_mapping)
train_df['Vehicle_Age'] = train_df['Vehicle_Age'].map(vehicle_age_mapping)
test_df['Vehicle_Age'] = test_df['Vehicle_Age'].map(vehicle_age_mapping)
train_df['Vehicle_Damage'] = train_df['Vehicle_Damage'].map(vehicle_damage_mapping)
test_df['Vehicle_Damage'] = test_df['Vehicle_Damage'].map(vehicle_damage_mapping)

train_df['Region_Code'] = train_df['Region_Code'].astype(int) 
train_df['Annual_Premium'] = train_df['Annual_Premium'].astype(int) 
train_df['Policy_Sales_Channel'] = train_df['Policy_Sales_Channel'].astype(int)
test_df['Region_Code'] = test_df['Region_Code'].astype(int) 
test_df['Annual_Premium'] = test_df['Annual_Premium'].astype(int) 
test_df['Policy_Sales_Channel'] = test_df['Policy_Sales_Channel'].astype(int)

In [5]:
train_df

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,0,0,21,1,35,0,1,1,65101,124,187,0
1,1,0,43,1,28,0,2,1,58911,26,288,1
2,2,1,25,1,14,1,0,0,38043,152,254,0
3,3,1,35,1,1,0,1,1,2630,156,76,0
4,4,1,36,1,15,1,1,0,31951,152,294,0
...,...,...,...,...,...,...,...,...,...,...,...,...
11504793,11504793,0,48,1,6,0,1,1,27412,26,218,0
11504794,11504794,1,26,1,36,0,0,1,29509,152,115,1
11504795,11504795,1,29,1,32,1,0,0,2630,152,189,0
11504796,11504796,1,51,1,28,0,1,1,48443,26,274,1


In [6]:
test_df

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
0,11504798,1,20,1,47,0,0,0,2630,160,228
1,11504799,0,47,1,28,0,1,1,37483,124,123
2,11504800,0,47,1,43,0,1,1,2630,26,271
3,11504801,1,22,1,47,1,0,0,24502,152,115
4,11504802,0,51,1,19,0,1,0,34115,124,148
...,...,...,...,...,...,...,...,...,...,...,...
7669861,19174659,0,57,1,28,0,1,1,51661,124,109
7669862,19174660,0,28,1,50,1,0,0,25651,152,184
7669863,19174661,0,47,1,33,1,1,0,2630,138,63
7669864,19174662,0,30,1,28,0,0,1,38866,124,119


In [7]:
X_train = train_df.drop('Response', axis=1)
y_train = train_df['Response']

In [8]:
categorical_data = ['Gender', 'Vehicle_Age', 'Vehicle_Damage', 'id', 'Region_Code', 'Annual_Premium', 'Policy_Sales_Channel']

model = CatBoostClassifier(
    learning_rate=0.1217, 
    loss_function = 'Logloss',
    eval_metric='F1',
    random_seed=37,
    logging_level='Silent',
    iterations=2000,
    task_type="GPU",
    devices='0'
)

In [9]:
model.fit(
    X_train, y_train,
    cat_features=categorical_data,
    logging_level='Silent'
)

In [10]:
predict = model.predict_proba(test_df)[:,1:].flatten()

submission = pd.DataFrame({
    'id': test_df['id'],
    'Response': predict
})

# submission = submission['Response'].apply(lambda item: threshold(item, hold))
submission.to_csv('kagg_base3.csv', index=False)

In [11]:
submission

,id,Response
0,11504798,0.012663
1,11504799,0.539683
2,11504800,0.224495
3,11504801,0.000086
4,11504802,0.095840
...,...,...
7669861,19174659,0.170596
7669862,19174660,0.000263
7669863,19174661,0.000315
7669864,19174662,0.643359


# Пример

In [ ]:
import random

import os

from tqdm.notebook import tqdm

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import roc_auc_score

import torch
import torch.nn as nn
from torch import optim
from torch.utils.data import DataLoader
from torch.nn.functional import relu, sigmoid

In [ ]:
!pip install torchview


In [ ]:
from torchview import draw_graph
import graphviz


In [ ]:
DEBUG = 1

PATH = '../input/playground-series-s4e7/'
DEVICE = 'cuda'


In [ ]:
seed = 1723

random.seed(seed)
np.random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True


In [ ]:
## data_colection

df = pd.read_csv(PATH+'train.csv')

print (df.shape)
print (df.isnull().any())
df.head()


In [ ]:
## data_split

if DEBUG:
    df = df.iloc[:100*1000]

df, test = train_test_split(df, 
                            test_size=.20)
df.reset_index(drop=True, 
               inplace=True)
test.reset_index(drop=True, 
                 inplace=True)


In [ ]:
## feats_engin

nums = [
    'Age', 
    'Annual_Premium', 
    'Vintage'
]
cats = [c for c in df.columns if c not in nums+['id', 'Response']]


In [ ]:
## feats_transform

for c in tqdm(cats):
    df[c] = f'{c}_'+df[c].astype(str)

uni = set()
for c in tqdm(cats):
    uni = uni | set(df[c])

tokens = dict(zip(uni, 
                  range(len(uni))))
for i,k in enumerate(tokens):
    print (k, tokens[k])
    if i == 5:
        break

df.head()


In [ ]:
## feats_treat

In [ ]:
## feats_select

In [ ]:
## model_trainer 

class Dataset:

    def __init__(self, 
                 df, 
                 nums, 
                 cats, 
                 tokens):
        self.df_ = df
        self.nums_ = nums
        self.cats_ = cats
        self.tokens_ = tokens

    def __len__(self):
        return self.df_.shape[0]

    def __getitem__(self, 
                    idx):
        xs = {
            'nums': torch.tensor([self.df_[num][idx] for num in self.nums_], 
                                 dtype=torch.float32), 
            'cats': torch.tensor([self.tokens_[self.df_[cat][idx]] for cat in self.cats_], 
                                 dtype=torch.int32)
        }
        ys = torch.tensor(self.df_['Response'][idx], 
                          dtype=torch.float32)

        return xs, ys

dataset = Dataset(df, 
                  nums, 
                  cats, 
                  tokens)
dataloader = DataLoader(dataset, 
                        batch_size=1024, 
                        shuffle=True, 
                        num_workers=0)

for xb, yb in dataloader:
    print (len(xb['nums']))
    print (xb['nums'])
    print (xb['cats'])
    print (yb)
    break


In [ ]:
## model_trainer 

model = nn.Linear(len(nums), 16)
model(xb['nums'])


In [ ]:
## model_trainer 

class NeuralNet(nn.Module):
    
    def __init__(self, 
                 n_nums, 
                 n_cats, 
                 evs_dim):
        super(NeuralNet, self).__init__()
        
        self.evs = nn.Embedding(evs_dim, 
                                100)
        
        self.linear1 = nn.Linear(100*2, 
                                 512)
        self.linear2 = nn.Linear(100*2, 
                                 512)
        
        self.bn3 = nn.BatchNorm1d(n_nums)
        self.linear3 = nn.Linear(n_nums, 
                                 512)
        
        self.bn4 = nn.BatchNorm1d(512*2)
        self.linear4 = nn.Linear(512*2, 
                                 512)
        self.dropout4 = nn.Dropout(0.2)
        
        self.out = nn.Linear(512, 1)
    
    def forward(self, 
                nums, 
                cats):
        
        h_evs = self.evs(cats)
        
        avg_pool = torch.mean(h_evs, 1)
        max_pool, _ = torch.max(h_evs, 1)
        h_concat = torch.cat((max_pool, 
                              avg_pool), 1)
        
        h_linear1  = relu(self.linear1(h_concat))
        h_linear2  = relu(self.linear2(h_concat))
        h_linear12 = h_linear1+h_linear2
        
        h_bn3 = self.bn3(nums)
        h_linear3  = relu(self.linear3(h_bn3))
        
        h_linear123 = torch.cat((h_linear12,
                                 h_linear3), 1)
        
        h_bn4 = self.bn4(h_linear123)
        h_linear4  = relu(self.linear4(h_bn4))
        h_dropout4 = self.dropout4(h_linear4)
        
        out = self.out(h_dropout4)
        
        return out

model = NeuralNet(len(nums), 
                  len(cats), 
                  max(tokens.values())+1)
graph = draw_graph(model,
                   input_data=(xb['nums'], xb['cats']), 
                   expand_nested=True)
graph.visual_graph


In [ ]:
## model_trainer 

def training_phase(tra, 
                   val, 
                   fold, 
                   nums, 
                   cats, 
                   tokens):

    train_dataset = Dataset(tra, 
                            nums, 
                            cats, 
                            tokens)
    train_dataloader = DataLoader(train_dataset, 
                                  batch_size=1024, 
                                  shuffle=True, 
                                  num_workers=0)

    valid_dataset = Dataset(val, 
                            nums, 
                            cats, 
                            tokens)
    valid_dataloader = DataLoader(valid_dataset, 
                                  batch_size=1024, 
                                  shuffle=False, 
                                  num_workers=0)

    model = NeuralNet(len(nums), 
                      len(cats), 
                      max(tokens.values())+1)
    model.to(DEVICE)

    loss_fn = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters())
    
    tr, best_loss = 0, np.inf
    for epoch in tqdm(range(32)):
        
        model.train()
        for xb, yb in train_dataloader:

            xb['nums'] = xb['nums'].to(DEVICE)
            xb['cats'] = xb['cats'].to(DEVICE)
            yb = yb.to(DEVICE)

            esti = torch.flatten(model(xb['nums'], 
                                       xb['cats']))

            loss = loss_fn(esti, yb)

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        
        model.eval()
        valid_loss = 0
        estimations = []
        with torch.no_grad():
            for xb, yb in valid_dataloader:

                xb['nums'] = xb['nums'].to(DEVICE)
                xb['cats'] = xb['cats'].to(DEVICE)
                yb = yb.to(DEVICE)
                
                esti = torch.flatten(model(xb['nums'], 
                                           xb['cats']))
                estimations += sigmoid(esti).tolist()

                loss = loss_fn(esti, yb)
                valid_loss += loss.item() / len(valid_dataloader)
        
        ms = roc_auc_score(val['Response'], 
                           estimations)
        print (f'epoch {epoch} statistics = [{valid_loss}, {ms}]')
        
        if valid_loss < best_loss:
            torch.save(model.state_dict(), 
                       f'model{fold}.pth')
            best_loss = valid_loss
            tr = 0
        else:
            tr += 1
            if tr == 4:
                break

kf = KFold(n_splits=5, 
           shuffle=True, 
           random_state=1723)
for fold, (tra_index, val_index) in enumerate(kf.split(df)):
    
    tra = df.iloc[tra_index]
    tra.reset_index(drop=True, 
                    inplace=True)
    
    val = df.iloc[val_index]
    val.reset_index(drop=True, 
                    inplace=True)
    
    training_phase(tra, 
                   val, 
                   fold, 
                   nums, 
                   cats, 
                   tokens)
    
    if DEBUG:
        break
